In [1]:
import pandas as pd
import requests
import json

In [182]:
url = 'https://api.hh.ru/vacancies'
params = {'text': 'оператор', 'per_page': '100', 'page': '1'}
src = requests.get(url, params=params)
data = src.json()

In [174]:
names = ['name', 'experience', 'employment', 'created_at', 'selary', 'address', 'type', 'schedule', 'work_schedule_by_days', 'working_hours']
df = pd.DataFrame(columns=names)

In [14]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

In [183]:
data['pages']

20

In [60]:
print(len(data['items']))


100


In [184]:
def sercher():
    for i in range(data['per_page']-1):
        val = data['items'][i]
        name = val['name']
        sel = (str(val['salary']['from']) + '-' + str(val['salary']['to']) + ' ' + str(val['salary']['currency'])) if val['salary'] != None else 'Не указано'
        exp = val['experience']['name']
        empl = val['employment']['name']
        create = val['created_at'][:10] + ' ' + val['created_at'][10:19]
        adr = val['address']['raw'] if val['address'] else 'Не указано'
        ono = val['type']['name']
        sch = val['schedule']['name']
        wsbd = val['work_schedule_by_days'][0]['name'] if val['work_schedule_by_days'] else 'Не указано'
        wh = val['working_hours'][0]['name'].replace('\xa0', ' ') if val['working_hours'] else 'Не указано'
        odata = [name, exp, empl, create, sel, adr, ono, sch, wsbd, wh]
        df.loc[val['id']] = odata

In [185]:
for x in range(1, data['pages']):
    params = {'text': 'оператор', 'per_page': '100', 'page': x}
    print(params['page'])
    src = requests.get(url, params=params)
    data = src.json()
    sercher()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [186]:
df.to_csv('Data_HHRu.csv', index=True)